In [1]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [2]:
import os
import shutil

src_folder = '/content/drive/MyDrive/Colab Notebooks/data/archive'
dst_folder = '/content/datasets'
os.makedirs(dst_folder, exist_ok=True)
print("Thư mục đích đã sẵn sàng:", dst_folder)

train_ratio = 0.8 # 80% dữ liệu để huấn luyện
valid_ratio = 0.1 # 10% dữ liệu để validation
img_size = 640 # Resize ảnh về 640x640

# Cấu trúc thư mục
for split in ['train', 'valid', 'test']:
    os.makedirs(os.path.join(dst_folder, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(dst_folder, split, 'labels'), exist_ok=True)

# Load class names
with open(os.path.join(src_folder, 'classes_vie.txt'), 'r', encoding='utf-8') as f:
    classes = [line.strip() for line in f.readlines()]

import random

all_images = [os.path.join(src_folder, 'images', f) for f in os.listdir(os.path.join(src_folder, 'images')) if f.endswith('.jpg')]
random.shuffle(all_images)

# Split dataset
train_size = int(len(all_images) * train_ratio)
valid_size = int(len(all_images) * valid_ratio)

train_images = all_images[:train_size]
valid_images = all_images[train_size:train_size+valid_size]
test_images = all_images[train_size+valid_size:]

# Comprehensive augmentation pipeline with YOLOv8
import albumentations as A

augmentations = A.Compose([
    A.Resize(640, 640),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomRotate90(p=0.3),
    A.Rotate(limit=15, p=0.5),
    A.Affine(shear=5, p=0.3),
    A.GaussianBlur(blur_limit=(3,5), p=0.2),
    A.RandomBrightnessContrast(p=0.5),
    A.RandomGamma(p=0.2),
    A.GaussNoise(p=0.3),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.3),
], bbox_params=A.BboxParams(format='yolo'))

Thư mục đích đã sẵn sàng: /content/datasets


In [3]:
import cv2
from tqdm import tqdm
import yaml

def process_and_save(images, split):
    for img_path in tqdm(images, desc=f'Processing {split}'):
        filename = os.path.basename(img_path)
        label_filename = filename.replace('.jpg', '.txt')

        # Read image and labels
        image = cv2.imread(img_path)
        h, w = image.shape[:2]

        label_path = os.path.join(src_folder, 'labels', label_filename)
        with open(label_path, 'r') as f:
            labels = [list(map(float, line.split())) for line in f.readlines()]

        bboxes = [label[1:] + [int(label[0])] for label in labels]  # bbox + class_id

        # Check bbox validity
        invalid_bbox = False
        for bbox in bboxes:
            x_center, y_center, bbox_w, bbox_h, _ = bbox
            if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= bbox_w <= 1 and 0 <= bbox_h <= 1):
                invalid_bbox = True
                break

        if invalid_bbox:
            print(f"Skipping {filename} due to invalid bbox values.")
            continue

        try:
            if split == 'train':
                # Save original image and labels
                original_img_resized = cv2.resize(image, (640, 640))
                cv2.imwrite(os.path.join(dst_folder, split, 'images', 'original_' + filename), original_img_resized)
                with open(os.path.join(dst_folder, split, 'labels', 'original_' + label_filename), 'w') as f:
                    for bbox in bboxes:
                        cls_id = bbox[-1]
                        bbox_coords = bbox[:4]
                        f.write(f"{cls_id} {' '.join(map(str, bbox_coords))}\n")

                # Save augmented image and labels
                augmented = augmentations(image=image, bboxes=bboxes)
                img_processed, bboxes_processed = augmented['image'], augmented['bboxes']
            else:
                img_processed = cv2.resize(image, (640, 640))
                bboxes_processed = bboxes
        except ValueError as e:
            print(f"Skipping {filename} due to augmentation error: {e}")
            continue

        # Save augmented image
        cv2.imwrite(os.path.join(dst_folder, split, 'images', filename), img_processed)

        # Save augmented labels
        with open(os.path.join(dst_folder, split, 'labels', label_filename), 'w') as f:
            for bbox in bboxes_processed:
                cls_id = bbox[-1]
                bbox_coords = bbox[:4]
                f.write(f"{cls_id} {' '.join(map(str, bbox_coords))}\n")

# Process all splits
process_and_save(train_images, 'train')
process_and_save(valid_images, 'valid')
process_and_save(test_images, 'test')

# Check dataset statistics
base_path = '/content/datasets'

for split in ['train', 'valid', 'test']:
    image_dir = os.path.join(base_path, split, 'images')
    label_dir = os.path.join(base_path, split, 'labels')

    num_images = len([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))])
    num_labels = len([f for f in os.listdir(label_dir) if f.endswith('.txt')])

    print(f'📁 {split.upper()}')
    print(f'Số ảnh:   {num_images}')
    print(f'Số nhãn:  {num_labels}\n')

# Create data.yaml for YOLOv8
data_yaml = {
    'path': '/content/datasets',  # Dataset root dir
    'train': 'train/images',      # Train images (relative to path)
    'val': 'valid/images',        # Val images (relative to path)
    'test': 'test/images',        # Test images (relative to path)
    'nc': len(classes),           # Number of classes
    'names': classes              # Class names
}

with open(os.path.join(dst_folder, 'data.yaml'), 'w', encoding='utf-8') as f:
    yaml.dump(data_yaml, f, allow_unicode=True, default_flow_style=False)

print("✅ Đã tạo file data.yaml cho YOLOv8")

Processing train:  85%|████████▌ | 2194/2572 [22:18<03:25,  1.84it/s]

Skipping 0248.jpg due to augmentation error: Expected x_max for bbox [ 0.96562546  0.294444    1.0187504   0.396296   49.        ] to be in the range [0.0, 1.0], got 1.0187504291534424.


Processing test: 100%|██████████| 323/323 [02:52<00:00,  1.87it/s]

📁 TRAIN
Số ảnh:   5143
Số nhãn:  5143

📁 VALID
Số ảnh:   321
Số nhãn:  321

📁 TEST
Số ảnh:   323
Số nhãn:  323

✅ Đã tạo file data.yaml cho YOLOv8


In [4]:
!wget https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov8m.pt

--2025-08-16 03:53:26--  https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov8m.pt
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/521807533/d860788d-065e-4444-bc38-4f0f997483de?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-16T04%3A46%3A14Z&rscd=attachment%3B+filename%3Dyolov8m.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-16T03%3A45%3A36Z&ske=2025-08-16T04%3A46%3A14Z&sks=b&skv=2018-11-09&sig=%2BC3i5AcjKA5z3q%2Bj3DIhrSMuztocF%2FY6Z4TYF5SVhEA%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NTMxNjcwNywibmJmIjoxNzU1MzE2NDA3LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iLmN

In [5]:

import torch
torch.cuda.empty_cache()
import torch
import os
from IPython.display import Image, clear_output

from ultralytics import YOLO

# Load YOLOv8 medium model
model = YOLO("/content/yolov8m.pt")

# Train the model with YOLOv8 parameters
results = model.train(
    data="/content/datasets/data.yaml",  # Path to data.yaml
    imgsz=640,                           # Image size
    epochs=50,                           # Number of epochs
    batch=16,                            # Batch size
    workers=2,                           # Number of workers
    device=0,                            # GPU device
    name='yolov8_training',              # Experiment name
    project='runs/detect',               # Project directory
    patience=10,                         # Early stopping patience
    save=True,                           # Save checkpoints
    verbose=True,                        # Verbose output
    optimizer='SGD',                     # Optimizer (SGD, Adam, AdamW)
    lr0=0.01,                           # Initial learning rate
    momentum=0.937,                      # Momentum
    weight_decay=0.0005,                # Weight decay
    warmup_epochs=3,                     # Warmup epochs
    warmup_momentum=0.8,                # Warmup momentum
    warmup_bias_lr=0.1,                 # Warmup bias learning rate
    box=7.5,                            # Box loss gain
    cls=0.5,                            # Class loss gain
    dfl=1.5,                            # DFL loss gain
    hsv_h=0.015,                        # HSV hue augmentation
    hsv_s=0.7,                          # HSV saturation augmentation
    hsv_v=0.4,                          # HSV value augmentation
    degrees=0.0,                        # Rotation degrees
    translate=0.1,                      # Translation
    scale=0.5,                          # Scale
    shear=0.0,                          # Shear
    perspective=0.0,                    # Perspective
    flipud=0.0,                         # Flip up-down
    fliplr=0.5,                         # Flip left-right
    mosaic=1.0,                         # Mosaic augmentation
    mixup=0.0,                          # Mixup augmentation
    copy_paste=0.0,                     # Copy-paste augmentation
)

print("🎉 Training completed!")
print(f"📁 Results saved to: {results.save_dir}")

# ==================== VALIDATION VÀ EXPORT ====================

# Validate the model
metrics = model.val()

# Export the model to different formats (optional)
# model.export(format='onnx')  # Export to ONNX
# model.export(format='torchscript')  # Export to TorchScript

print("✅ Model validation completed!")
print(f"📊 mAP50: {metrics.box.map50}")
print(f"📊 mAP50-95: {metrics.box.map}")

# ==================== INFERENCE EXAMPLE ====================

# Load best model for inference
best_model = YOLO(f'{results.save_dir}/weights/best.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, lin

Overriding model.yaml nc=80 with nc=52

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             


  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384, 576, 3, 2]              
  8                  -1  2   3985920  ultralytics.nn.modules.block.C2f             [576, 576, 2, True]           
  9                  -1  1    831168  ultralytics.nn.modules.block.SPPF            [576, 576, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1612.0±889.1 MB/s, size: 266.9 KB)


train: Scanning /content/datasets/train/labels... 5143 images, 46 backgrounds, 0 corrupt: 100%|██████████| 5143/5143 [00:02<00:00, 2223.71it/s]


...


50 epochs completed in 2.526 hours.
Optimizer stripped from runs/detect/yolov8_training/weights/last.pt, 52.1MB
Optimizer stripped from runs/detect/yolov8_training/weights/best.pt, 52.1MB

Validating runs/detect/yolov8_training/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,869,868 parameters, 0 gradients, 78.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:07<00:00,  1.49it/s]


                   all        321        837      0.937      0.968      0.982      0.812
Đường người đi bộ cắt ngang         37         37      0.993      0.973      0.993      0.787
Đường giao nhau (ngã ba bên phải)          2          2      0.931          1      0.995      0.456
    Cấm đi ngược chiều         47         47      0.999      0.979      0.995      0.777
Phải đi vòng sang bên phải         47         47      0.978       0.96       0.99      0.747
Giao nhau với đường đồng cấp          5          5          1          1      0.995      0.718
Giao nhau với đường không ưu tiên         29         29          1      0.959      0.995      0.767
Chỗ ngoặt nguy hiểm vòng bên trái         14         14          1      0.983      0.995      0.857
           Cấm rẽ trái         28         28      0.993          1      0.995      0.826
           Bến xe buýt         21         21        0.9      0.952      0.979      0.815
Nơi giao nhau chạy theo vòng xuyến          1          1      

val: Scanning /content/datasets/valid/labels.cache... 321 images, 1 backgrounds, 0 corrupt: 100%|██████████| 321/321 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:09<00:00,  2.15it/s]


                   all        321        837      0.937      0.968      0.982      0.814
Đường người đi bộ cắt ngang         37         37      0.993      0.973      0.993      0.788
Đường giao nhau (ngã ba bên phải)          2          2      0.931          1      0.995      0.456
    Cấm đi ngược chiều         47         47      0.999      0.979      0.995      0.768
Phải đi vòng sang bên phải         47         47      0.978       0.96      0.991       0.75
Giao nhau với đường đồng cấp          5          5          1          1      0.995      0.749
Giao nhau với đường không ưu tiên         29         29          1       0.96      0.995      0.759
Chỗ ngoặt nguy hiểm vòng bên trái         14         14          1      0.983      0.995      0.862
           Cấm rẽ trái         28         28      0.993          1      0.995      0.806
           Bến xe buýt         21         21        0.9      0.952      0.976      0.825
Nơi giao nhau chạy theo vòng xuyến          1          1      

In [6]:
# Nén thư mục runs thành file zip
!zip -r runs.zip runs

# Tải file zip về máy
from google.colab import files
files.download("runs.zip")


  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/yolov8_training2/ (stored 0%)
  adding: runs/detect/yolov8_training2/BoxF1_curve.png (deflated 9%)
  adding: runs/detect/yolov8_training2/val_batch2_pred.jpg (deflated 6%)
  adding: runs/detect/yolov8_training2/val_batch2_labels.jpg (deflated 6%)
  adding: runs/detect/yolov8_training2/BoxPR_curve.png (deflated 21%)
  adding: runs/detect/yolov8_training2/val_batch0_pred.jpg (deflated 8%)
  adding: runs/detect/yolov8_training2/confusion_matrix.png (deflated 15%)
  adding: runs/detect/yolov8_training2/BoxP_curve.png (deflated 12%)
  adding: runs/detect/yolov8_training2/val_batch1_pred.jpg (deflated 6%)
  adding: runs/detect/yolov8_training2/val_batch1_labels.jpg (deflated 6%)
  adding: runs/detect/yolov8_training2/confusion_matrix_normalized.png (deflated 15%)
  adding: runs/detect/yolov8_training2/BoxR_curve.png (deflated 11%)
  adding: runs/detect/yolov8_training2/val_batch0_labels.jpg (deflated 8%)
  a

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>